# Projet Python - ENSAE 2A
Manon Vallat, Clément Hadji, Iris Lafaille - Sous la direction de Romain Avouac

### Sujet
Analyse de l'innovation : l'effet du dépôt de brevet sur la côte en bourse 

### Objectifs

1. Collecter et analyser les données de brevets
2. 
3. 
4. 


### Données
Source : 

### Méthodologie

1. Extraction des données
2. Nettoyage et prétraitement
3. Analyse exploratoire
4. Visualisations
5. Conclusions

## Introduction

## Sommaire